<a href="https://colab.research.google.com/github/agarwalanant/FyndImageSegmentation/blob/master/MaskGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using Keras for rapid prototyping, if time permits fimal model will be implemented in pytorch or TF 2.0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


**AutoEncoder and Decoder for gerating masks**

In [0]:
!wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip
!unzip PennFudanPed.zip -d /content/drive/My\ \Drive/imagesegmentdata/Penn_Fudan

In [0]:
!wget http://alphamatting.com/datasets/zip/input_training_lowres.zip
!wget http://alphamatting.com/datasets/zip/trimap_training_lowres.zip
!wget http://alphamatting.com/datasets/zip/gt_training_lowres.zip    

In [0]:
!unzip input_training_lowres.zip -d /content/drive/My\ \Drive/imagesegmentdata/alphamatting/input

In [0]:
# !unzip trimap_training_lowres.zip -d /content/drive/My\ \Drive/imagesegmentdata/alphamatting
!unzip gt_training_lowres.zip  -d /content/drive/My\ \Drive/imagesegmentdata/alphamatting/gt

In [0]:
#model parameters
batch_size = 1
epochs = 2
inChannel = 3
iml, imh = 500, 500
input_img = Input(shape = (iml, imh, inChannel))
num_classes = 1
batch = 1

In [0]:
# Data Generators
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


In [34]:
train_generator=train_datagen.flow_from_directory('drive/My Drive/imagesegmentdata/alphamatting/input',
                                                 target_size=(iml,imh),
                                                 color_mode='rgb',
                                                 batch_size=batch,
                                                 class_mode='input',
                                                 shuffle=True,)

val_datagen = ImageDataGenerator(rescale=1. /255)
val_generator=train_datagen.flow_from_directory('drive/My Drive/imagesegmentdata/alphamatting/input',
                                                 target_size=(iml,imh),
                                                 color_mode='rgb',
                                                 batch_size=batch,
                                                 class_mode='input',
                                                 shuffle=True)

Found 27 images belonging to 1 classes.
Found 27 images belonging to 1 classes.


In [0]:
def encoder(input_img):
    #encoder

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 256 (small and thick)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    return conv4



In [0]:
def decoder(conv4):    
    #decoder
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4) #7 x 7 x 128
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded

In [0]:
autoencoder = Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())

In [48]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 500, 500, 3)       0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 500, 500, 32)      896       
_________________________________________________________________
batch_normalization_43 (Batc (None, 500, 500, 32)      128       
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 500, 500, 32)      9248      
_________________________________________________________________
batch_normalization_44 (Batc (None, 500, 500, 32)      128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 250, 250, 32)      0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 250, 250, 64)      18496     
__________

In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=8)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, mode='auto')
checkpointer = ModelCheckpoint('/content/drive/My Drive/imagesegmentdata/alphamatting/model.hdf5', mode='min', monitor='val_loss', verbose=1, save_best_only=True)


batch_size = batch
learning_rate = 0.001

In [50]:
# checkpoint = ModelCheckpoint('gdrive/My Drive/101/{epoch:02d}-{val_acc:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]
step_size_train=train_generator.n//train_generator.batch_size
step_size_val=val_generator.n//val_generator.batch_size
model_history = []
model_history.append(autoencoder.fit_generator(generator=train_generator,
                    validation_data=val_generator,
                   steps_per_epoch=step_size_train,
                                               
                    validation_steps = step_size_val,
                                               
                 epochs=epochs,shuffle = True,
          callbacks=[lr_reducer, checkpointer, early_stopper],initial_epoch =0))

Epoch 1/2
27/27 [==============================] - 233s 9s/step - loss: 0.0630 - val_loss: 0.0738

Epoch 00001: val_loss improved from inf to 0.07376, saving model to /content/drive/My Drive/imagesegmentdata/alphamatting/model.hdf5
Epoch 2/2
27/27 [==============================] - 227s 8s/step - loss: 0.0378 - val_loss: 0.3652

Epoch 00002: val_loss did not improve from 0.07376


In [0]:
import cv2
img = cv2.imread('/content/drive/My Drive/imagesegmentdata/alphamatting/input/input/GT01.png')
img = cv2.resize(img,(500,500))
img = np.reshape(img,[1,500,500,3])
ans = autoencoder.predict(img)
ans = ans.reshape(500,500,3)
cv2.imwrite('/content/drive/My Drive/imagesegmentdata/alphamatting/t.png',ans)